In [2]:
from parameterized.proposal_inverter import Wallet, ProposalInverter

import numpy as np
rng = np.random.default_rng(42)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [6]:
# Deploy 10 wallets & add random values to funds & wallet feature matrix entries
WALLETS = dict()
number_of_wallets = 10
total_funds = 1000

funds_distribution = rng.random(size=number_of_wallets)
funds_distribution = funds_distribution / np.sum(funds_distribution) * total_funds

for funds in funds_distribution:
    wallet = Wallet(funds=funds)
    no_features = wallet.number_of_features
    # fill in wallet feature matrix (5 features) with random values
    wallet.wallet_feature_matrix = list(rng.random(size=wallet.number_of_features))
    WALLETS[wallet.public] = wallet

In [8]:
"""Get the wallet matrix by appending every wallet_feature_matrix to a 2D array
every entry in the array is an array respective to each wallet's preference of a 
certain feature. It is a number within range (0-1), the closer the rating is to
 1, the higher the preference"""

wallet_matrix = []
for wallet in WALLETS.values():
    wallet_matrix.append(wallet.wallet_feature_matrix)
wallet_matrix = np.asmatrix(wallet_matrix)

"""The matrix is represented by the 2d array below, where the rows represent
wallets and the columns represent features"""

wallet_matrix

matrix([[0.37079802, 0.92676499, 0.64386512, 0.82276161, 0.4434142 ],
        [0.22723872, 0.55458479, 0.06381726, 0.82763117, 0.6316644 ],
        [0.75808774, 0.35452597, 0.97069802, 0.89312112, 0.7783835 ],
        [0.19463871, 0.466721  , 0.04380377, 0.15428949, 0.68304895],
        [0.74476216, 0.96750973, 0.32582536, 0.37045971, 0.46955581],
        [0.18947136, 0.12992151, 0.47570493, 0.22690935, 0.66981399],
        [0.43715192, 0.8326782 , 0.7002651 , 0.31236664, 0.8322598 ],
        [0.80476436, 0.38747838, 0.2883281 , 0.6824955 , 0.13975248],
        [0.1999082 , 0.00736227, 0.78692438, 0.66485086, 0.70516538],
        [0.78072903, 0.45891578, 0.5687412 , 0.139797  , 0.11453007]])

In [9]:
PROPOSALS = dict()
number_of_proposals = 20
owner_wallet = Wallet(funds=5000)
deploy_funds = owner_wallet.funds/number_of_proposals

for i in range(number_of_proposals):
    proposal = owner_wallet.deploy(deploy_funds)
    # fill in proposal feature matrix (5 features) with random values
    proposal.proposal_feature_matrix = list(rng.random(size=proposal.number_of_features))
    PROPOSALS[proposal.public] = proposal

In [10]:
"""Get the proposal matrix by appending every proposal_feature_matrix to a 2D array
every entry in the array is an array respective to the degree of presence of a 
certain feature in a proposal. It is a number within range (0-1), the closer the 
rating is to 1, the higher the preference"""

proposal_matrix = []
for proposal in PROPOSALS.values():
    proposal_matrix.append(proposal.proposal_feature_matrix)
proposal_matrix = np.asmatrix(proposal_matrix)
proposal_matrix = np.matrix.transpose(proposal_matrix)

"""The matrix is represented by the 2d array below, where the rows represent
features and the columns represent proposals. We collect the transpose of the
matrix in order to successfully multiply the wallet matrix by the proposal matrix"""

proposal_matrix

matrix([[0.66840296, 0.5535794 , 0.21458467, 0.28138389, 0.66431354,
         0.09004786, 0.1523121 , 0.63028259, 0.90858069, 0.71689019,
         0.45577629, 0.85661428, 0.58409797, 0.49399082, 0.17059297,
         0.02280387, 0.48665833, 0.26697566, 0.82629192, 0.67224009],
        [0.47109621, 0.55920716, 0.40852864, 0.29359376, 0.40638686,
         0.72235935, 0.69632038, 0.36181261, 0.69970713, 0.4493615 ,
         0.20236336, 0.75851953, 0.6498466 , 0.32986121, 0.92512012,
         0.95855921, 0.49070699, 0.331569  , 0.89616077, 0.28123378],
        [0.56523611, 0.3039501 , 0.85340307, 0.66191651, 0.81402038,
         0.46187723, 0.44615628, 0.08764992, 0.26586996, 0.27224156,
         0.30595662, 0.71946296, 0.08444432, 0.14452419, 0.58106114,
         0.48230344, 0.93782645, 0.5206724 , 0.14024909, 0.65942263],
        [0.76499886, 0.03081783, 0.23393949, 0.55703215, 0.16697292,
         0.16127178, 0.38102123, 0.1180059 , 0.96917638, 0.09639096,
         0.57921957, 0.43209304

In [11]:
"""The product of both matrices is a matrix where the entries represent
the likelihood for a wallet to fund a proposal. The higher the number
, the more likely a wallet will fund a proposal"""

wallet_proposal_matrix = np.matmul(wallet_matrix, proposal_matrix)
wallet_proposal_matrix

matrix([[1.95922859, 1.13822506, 1.22598278, 1.60851023, 1.29452036,
         1.35509216, 1.43625122, 1.14906733, 2.29925872, 1.33709432,
         1.10848222, 2.11750454, 1.23377089, 0.92757467, 1.84216095,
         1.88804126, 1.91940357, 1.11222599, 1.73120303, 1.87345463],
        [1.48328751, 0.75668435, 0.56023094, 1.22518341, 0.58082068,
         0.9005133 , 0.95505186, 1.05473583, 1.90550854, 1.07940594,
         0.82636422, 1.41509581, 0.86893508, 0.76118666, 1.24924514,
         1.26763723, 1.21484212, 0.6546866 , 1.22083398, 1.43802016],
        [2.3996884 , 1.28041664, 1.39022289, 2.06759195, 1.60465637,
         1.30674286, 1.37040275, 1.54528191, 2.66668642, 1.75569995,
         1.36916322, 2.4908832 , 1.1589131 , 1.18148631, 1.79109406,
         1.58876549, 2.33242311, 1.23418005, 1.6595885 , 2.49701867],
        [0.92630202, 0.68511009, 0.34573568, 0.84217407, 0.3959032 ,
         0.74201969, 0.63891159, 0.97061259, 1.19651781, 0.99257997,
         0.40667295, 1.04741254